# M14a - Multimodales RAG Enhanced

Dieses Notebook zeigt die Verwendung der erweiterten RAG-Funktionen mit automatischen Bildbeschreibungen.

**Neue Features:**
- Automatische Bildbeschreibungen mit GPT-4o-mini
- Cross-References zwischen Text und Bildern
- Cross-Modal-Retrieval
- Hybride Suche über Text und Bildbeschreibungen

In [1]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/Python_Modules
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint, install_packages
setup_api_keys(['OPENAI_API_KEY', 'HF_TOKEN'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()

✓ OPENAI_API_KEY erfolgreich gesetzt
✓ HF_TOKEN erfolgreich gesetzt

Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]

Installierte LangChain-Bibliotheken:
langchain                                0.3.27
langchain-community                      0.3.31
langchain-core                           0.3.78
langchain-experimental                   0.3.4
langchain-ollama                         0.3.10
langchain-openai                         0.3.35
langchain-text-splitters                 0.3.11

IP-Adresse: 35.227.3.113
Hostname: 113.3.227.35.bc.googleusercontent.com
Stadt: North Charleston
Region: South Carolina
Land: US
Koordinaten: 32.8546,-79.9748
Provider: AS396982 Google LLC
Postleitzahl: 29415
Zeitzone: America/New_York


In [2]:
#@title 🛠️ Installationen { display-mode: "form" }
install_packages([
    ('markitdown[all]', 'markitdown'),
    'langchain_chroma',
])

🔄 Installiere markitdown[all]...
✅ markitdown[all] erfolgreich installiert und importiert
🔄 Installiere langchain_chroma...
✅ langchain_chroma erfolgreich installiert und importiert


In [3]:
#@title 📂 Dokumente & Bilder kopieren { display-mode: "form" }
!rm -rf files
!mkdir files

# --- Texte
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_1.txt -o files/biografien_1.txt
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_2.md -o files/biografien_2.md
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_3.pdf -o files/biografien_3.pdf
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_4.docx -o files/biografien_4.docx
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/roboter.txt -o files/roboter.txt

# --- Bilder
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/a_retro-futuristic_robot_dall_e.jpg -o files/a_retro-futuristic_robot_dall_e.jpg
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/hedra_cyborg.png -o files/hedra_cyborg.png
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/apfel.jpg -o files/apfel.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5324  100  5324    0     0  17934      0 --:--:-- --:--:-- --:--:-- 17986
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7868  100  7868    0     0  46862      0 --:--:-- --:--:-- --:--:-- 47113
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 68184  100 68184    0     0   187k      0 --:--:-- --:--:-- --:--:--  187k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18743  100 18743    0     0  61269      0 --:--:-- --:--:-- --:--:-- 61452
  % Total    % Received % Xferd  Average Speed   Tim

## 2. Enhanced Funktionen laden

In [20]:
import M14a_standalone as m14

In [22]:
help(m14)

Help on module M14a_standalone:

NAME
    M14a_standalone

DESCRIPTION
    M14a - Standalone Multimodales RAG mit Bildbeschreibungen
    Vollständig eigenständige Version - benötigt KEIN M14.ipynb

    Verwendung:
        from M14a_standalone import *

        # System initialisieren
        rag = init_rag_system_enhanced()

        # Verzeichnis verarbeiten
        process_directory(rag, './files', auto_describe_images=True)

        # Suchen
        result = multimodal_search(rag, "Roboter")

    Autor: Enhanced by Claude
    Datum: Oktober 2025

CLASSES
    builtins.object
        RAGComponents
        RAGConfig

    class RAGComponents(builtins.object)
     |  RAGComponents(text_embeddings: langchain_openai.embeddings.base.OpenAIEmbeddings, clip_model: sentence_transformers.SentenceTransformer.SentenceTransformer, llm: langchain_openai.chat_models.base.ChatOpenAI, vision_llm: langchain_openai.chat_models.base.ChatOpenAI, text_splitter: langchain_text_splitters.character.RecursiveCh

In [31]:
# 1. (Import statement removed - functions should be available from %run M14_Multimodal_RAG.ipynb)
from pathlib import Path

# 2. Initialisierung
rag_components = m14.init_rag_system_enhanced()

# Dokumente/Bilder verarbeiten
process_directory(rag_components, './files', auto_describe_images=True)

# Suchen
result = multimodal_search(rag_components, "Apfel mit hellem neutralen Hintergrund")
print(result)


🔍 Multimodale Suche: Apfel mit hellem neutralen Hintergrund

📄 TEXT-ERGEBNISSE:
----------------------------------------------------------------------
Das Bild zeigt einen einzelnen, glänzenden roten Apfel auf einem neutralen, hellen Hintergrund. Die lebhaften Farben und die einfache Komposition heben den Apfel als Hauptobjekt hervor.

🖼️ Relevante Bilder (3):    • apfel.jpg (Ähnlichkeit: 1)
   • a_retro-futuristic_robot_dall_e.jpg (Ähnlichkeit: 0.902)
   • hedra_cyborg.png (Ähnlichkeit: 0.885)

🖼️ BILD-ERGEBNISSE via CLIP (1 gefunden):
----------------------------------------------------------------------
   1. apfel.jpg (Ähnlichkeit: 0.272)
      📝 Das Bild zeigt einen einzelnen, glänzenden roten Apfel, der auf einem neutralen, hellen Hintergrund platziert ist. Die Farben sind lebhaft, mit versch...

🔗 CROSS-MODAL RETRIEVAL (3 Bilder via Textsuche):
----------------------------------------------------------------------
   1. apfel.jpg
      📝 Das Bild zeigt einen einzelnen, glänzend

## 1. Setup
Zuerst führen wir das Original-Notebook M14 bis Sektion 2 aus.

In [ ]:
%run M14_Multimodal_RAG.ipynb

## 3. Enhanced RAG-System initialisieren

In [ ]:
# Initialisiere mit Vision-LLM Support
rag_components = init_enhanced_rag_system()

## 4. Bildbeschreibung testen

In [ ]:
# Teste Bildbeschreibung für ein einzelnes Bild
test_image = './files/apfel.jpg'

if Path(test_image).exists():
    description = generate_image_description(rag_components.vision_llm, test_image)
    print(f'Beschreibung: {description}')
else:
    print(f'Bild nicht gefunden: {test_image}')

## 5. Verzeichnis mit automatischen Bildbeschreibungen verarbeiten

In [ ]:
# Verarbeite alle Dateien mit automatischen Bildbeschreibungen
results = process_directory_enhanced(
    rag_components,
    './files',
    include_images=True,
    auto_describe_images=True
)

print(f'Text-Dokumente: {results["texts"]}')
print(f'Bilder: {results["images"]}')
print(f'Bildbeschreibungen: {results["image_descriptions"]}')

## 6. Erweiterte Textsuche (inkl. Bildbeschreibungen)

In [ ]:
# Textsuche findet nun auch Bildbeschreibungen
query = 'Roboter'
result = search_texts_enhanced(rag_components, query, k=5)
print(result)

## 7. Multimodale Suche mit Cross-Modal-Retrieval

In [ ]:
# Kombinierte Suche über alle Modalitäten
queries = ['Roboter', 'Cyborg', 'Apfel']

for query in queries:
    print(f'
=== Query: {query} ===')
    result = multimodal_search_enhanced(
        rag_components,
        query,
        k_text=3,
        k_images=3,
        enable_cross_modal=True
    )
    print(result)

## 8. Cross-Reference-Analyse

In [ ]:
# Analysiere Cross-References zwischen Collections
all_texts = rag_components.text_collection.get()
image_descriptions = [
    (id, meta) for id, meta in zip(all_texts['ids'], all_texts['metadatas'])
    if meta.get('doc_type') == 'image_description'
]

print(f'Gefundene Bildbeschreibungen: {len(image_descriptions)}')

for text_id, meta in image_descriptions[:3]:
    print(f"
{meta.get('filename', 'Unbekannt')}:")
    print(f"  Text-ID: {text_id}")
    print(f"  Bild-ID: {meta.get('image_doc_id', 'N/A')}")
    print(f"  Beschreibung: {meta.get('description', 'N/A')[:100]}...")

## 9. Vergleich: Original vs. Enhanced

### Vorteile der Enhanced Version:

1. **Semantische Bildsuche**: Textanfragen finden automatisch relevante Bilder
2. **Bessere Trefferquote**: Bildbeschreibungen erhöhen Recall
3. **Einheitliche Suche**: Eine Query durchsucht Text und Bilder
4. **Explainable AI**: Beschreibungen machen Ergebnisse nachvollziehbar
5. **Cross-Modal Intelligence**: System versteht Zusammenhänge